## 国土数値情報 観光資源データ
2023/04/25  
データを取得し、分析しやすい形式に変換する  
[国土交通省　観光資源データ - 国土数値情報](https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-P12-v2_2.html)

### データクレンジング
P12-14_22.xml：静岡県のデータ

In [1]:
import pandas as pd
import xmljson
from lxml import etree
import os
import csv
import collections

In [98]:
!pip install xmljson

In [2]:
#xmlデータを読み込みます

i = 22
name = 'P12-14_{:02}'.format(i)
if i in [12,17,21,22,28]:
    xml_tree = etree.parse('01_input/{}_replace.xml'.format(name))
else:
    xml_tree = etree.parse('01_input/{}.xml'.format(name))
# すべてのタグの取得
xml_root = xml_tree.getroot()
# xmlデータをdict型に変換
xml_dict = xmljson.yahoo.data(xml_root)


In [3]:
if not os.path.exists('01_output/{}'.format(name)):
    # ディレクトリが存在しない場合、ディレクトリを作成する
    os.makedirs('01_output/{}'.format(name))

In [4]:
xml_keys = xml_dict['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}Dataset'].keys()
xml_keys

odict_keys(['{http://www.opengis.net/gml/3.2}id', '{http://www.opengis.net/gml/3.2}description', '{http://www.opengis.net/gml/3.2}boundedBy', '{http://www.opengis.net/gml/3.2}Surface', '{http://www.opengis.net/gml/3.2}Curve', '{http://www.opengis.net/gml/3.2}Point', '{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}TourismResource_Surface', '{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}TourismResource_Line', '{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}TourismResource_Point'])

In [5]:
#データセットid
xml_dict['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}Dataset']['{http://www.opengis.net/gml/3.2}id']

'P12Dataset'

In [6]:
#データセット名
xml_dict['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}Dataset']['{http://www.opengis.net/gml/3.2}description']

'国土数値情報 観光資源（面） インスタンス文書'

In [7]:
#データセット基本情報
xml_dict['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}Dataset']['{http://www.opengis.net/gml/3.2}boundedBy']

OrderedDict([('{http://www.opengis.net/gml/3.2}EnvelopeWithTimePeriod',
              OrderedDict([('srsName', 'JGD2000 / (B, L)'),
                           ('frame', 'GC / JST'),
                           ('{http://www.opengis.net/gml/3.2}lowerCorner',
                            '20.0 123.0'),
                           ('{http://www.opengis.net/gml/3.2}upperCorner',
                            '46.0 154.0'),
                           ('{http://www.opengis.net/gml/3.2}beginPosition',
                            OrderedDict([('calendarEraName', '西暦'),
                                         ('content', '1900')])),
                           ('{http://www.opengis.net/gml/3.2}endPosition',
                            OrderedDict([('indeterminatePosition',
                                          'unknown')]))]))])

In [8]:
def getCurve(name, xmldata):
    #★観光資源（線）-面にあるIDがデータになっている　緯度経度
    #xml_dict['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}Dataset']['{http://www.opengis.net/gml/3.2}Curve']
    for i in range(len(xmldata)):
        xml_id = xmldata[i]['{http://www.opengis.net/gml/3.2}id']
        #print(xml_id)
        if 'pc_' in xml_id:
            xml_id = 'pa'+xml_id[2:]
        savelatlon(name, xml_id, xmldata[i]['{http://www.opengis.net/gml/3.2}segments']['{http://www.opengis.net/gml/3.2}LineStringSegment'])

In [9]:
def savelatlon(name, xml_id, xmldata):
    if isinstance(xmldata, collections.OrderedDict):
        data = xmldata['{http://www.opengis.net/gml/3.2}posList']
        data = data.split('\n')[1:-1]
        data = [[data[i].split(' ')[0], data[i].split(' ')[1]] for i in range(len(data))]
    else:
        data = []
        for i in range(len(xmldata)):
            xdata = xmldata[i]['{http://www.opengis.net/gml/3.2}posList']
            xdata = xdata.split('\n')[1:-1]
            data = data + [[xdata[i].split(' ')[0], xdata[i].split(' ')[1]] for i in range(len(xdata))]
    f = open('01_output/{}/{}.csv'.format(name, xml_id), 'w')
    writer = csv.writer(f)
    writer.writerows(data)
    f.close()

In [10]:
def multicode(xmldata):
    text = ''
    if isinstance(xmldata, collections.OrderedDict):#len(xmldata) == 1
        return xmldata['content']
    else:
        for i in range(len(xmldata)):
            if i > 0:
                text = text + ','
            text = text + xmldata[i]['content']
        return text

In [11]:
def getResource(resources, name, xmldata):
    if isinstance(xmldata, collections.OrderedDict):
        xml_id = xmldata['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}'+'{}'.format(getflag(xmldata['{http://www.opengis.net/gml/3.2}id']))]['{http://www.w3.org/1999/xlink}href'][1:]
        latlogs = pd.read_csv('01_output/{}/{}.csv'.format(name, xml_id),header=None)
        resources.append([xml_id,xmldata['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}turismResorceName'],xmldata['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}address'], latlogs.mean()[0], latlogs.mean()[1],multicode(xmldata['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}prefectureCode']),multicode(xmldata['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}administartiveAreaCode']),xmldata['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}turismResorceKindName'],xmldata['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}tourismResourceCategoryCode']])
    else:
        flag = getflag(xmldata[0]['{http://www.opengis.net/gml/3.2}id'])
        for i in range(len(xmldata)):
            xml_id = xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}'+'{}'.format(flag)]['{http://www.w3.org/1999/xlink}href'][1:]
            latlogs = pd.read_csv('01_output/{}/{}.csv'.format(name, xml_id),header=None)
            resources.append([xml_id,xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}turismResorceName'],xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}address'], latlogs.mean()[0], latlogs.mean()[1],multicode(xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}prefectureCode']),multicode(xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}administartiveAreaCode']),xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}turismResorceKindName'],xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}tourismResourceCategoryCode']])
            #print([xml_id,xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}turismResorceName'],xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}address'], latlogs.mean()[0], latlogs.mean()[1],xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}prefectureCode']['content'],xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}administartiveAreaCode']['content'],xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}turismResorceKindName'],xmldata[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}tourismResourceCategoryCode']])
    return resources

In [12]:
def getflag(gmlid):
    if 'FL03_' in gmlid:
        flag = 'location'
    elif 'FL02_' in gmlid:
        flag = 'bounds'
    #elif 'FL04_' in gmlid:
    #    flag = 'position'
    return flag

In [13]:
def getPointLatlon(pos):
    if isinstance(pos, list):
        plist = [[],[]]
        for i in range(len(pos)):
            plist[0].append(pos[i].split(' ')[0])
            plist[1].append(pos[i].split(' ')[1])
        return plist
    else:
        return [float(s) for s in pos.split(' ')]

In [14]:
def getResourcePoint(resources, xmldata1, xmldata2):
    dict_points = {}
    for i in range(len(xmldata1)):
        dict_points[xmldata1[i]['{http://www.opengis.net/gml/3.2}id']] = getPointLatlon(xmldata1[i]['{http://www.opengis.net/gml/3.2}pos'])
    #print(dict_points)
    for i in range(len(xmldata2)):
        xml_id = xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}position']['{http://www.w3.org/1999/xlink}href'][1:]
        if isinstance(dict_points[xml_id][0], list):
            for j in range(len(dict_points[xml_id][0])):
                resources.append([xml_id, xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}turismResorceName'], xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}address'], dict_points[xml_id][0][j], dict_points[xml_id][1][j],multicode(xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}prefectureCode']),multicode(xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}administartiveAreaCode']),xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}turismResorceKindName'],xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}tourismResourceCategoryCode']])
        else:
            resources.append([xml_id, xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}turismResorceName'], xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}address'], dict_points[xml_id][0], dict_points[xml_id][1],multicode(xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}prefectureCode']),multicode(xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}administartiveAreaCode']),xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}turismResorceKindName'],xmldata2[i]['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}tourismResourceCategoryCode']])
    return resources

In [15]:
resources = []

In [16]:
#面と線の緯度経度情報の取得
getCurve(name, xml_dict['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}Dataset']['{http://www.opengis.net/gml/3.2}Curve'])

In [17]:
#観光資源（線）
if '{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}TourismResource_Line' in xml_keys:
    resources = getResource(resources, name, xml_dict['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}Dataset']['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}TourismResource_Line'])
    resources

In [18]:
#観光資源（面）に関する情報 名前、住所
if '{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}TourismResource_Surface' in xml_keys:
    resources = getResource(resources, name, xml_dict['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}Dataset']['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}TourismResource_Surface'])
    resources

In [19]:
if '{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}TourismResource_Point' in xml_keys:
    resources = getResourcePoint(resources, xml_dict['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}Dataset']['{http://www.opengis.net/gml/3.2}Point'], xml_dict['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}Dataset']['{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}TourismResource_Point'])

In [20]:
pd_resources = pd.DataFrame(resources, columns=['id', 'name', 'adress', 'lat', 'lon', 'precode', 'areacode', 'category', 'category_flag'])
pd_resources.to_csv('01_output/{}/resources.csv'.format(name))
pd_resources.to_excel('01_output/{}/resources.xlsx'.format(name))
pd_resources

<ipython-input-20-0d2d0370a69b>:3: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  pd_resources.to_excel('01_output/{}/resources.xlsx'.format(name))


,id,name,adress,lat,lon,precode,areacode,category,category_flag
0,lc_00000_0,瓜生羅漢渓谷,相生市矢野町瓜生字羅漢口28,34.882238,134.443997,28,28208,‐,1
1,lc_00001_0,赤西・音水渓谷,宍粟市波賀町原 および 音水,35.235281,134.519099,28,28227,‐,1
2,lc_00002_0,立雲峡,朝来市和田山町竹田,35.286338,134.839249,28,28225,‐,1
3,lc_00003_0,千種川他,佐用町千種川（周辺）,35.035282,134.407736,28,28501,‐,1
4,pa_00000_0,須磨離宮公園,神戸市須磨区東須磨1-1,34.654103,135.118938,28,28107,‐,4
...,...,...,...,...,...,...,...,...,...
1008,n00869,加藤文太郎記念図書館,新温泉町浜坂842-2,35.624568,134.45453,28,28586,‐,2
1009,n00870,七釜,新温泉町七釜,35.609398,134.475908,28,28586,‐,3
1010,n00871,買い物ツアー,新温泉町,35.623469,134.448971,28,28586,‐,5
1011,n00872,以命亭,新温泉町浜坂1208,35.625059,134.45353,28,28586,‐,2


## メモ：XMLファイル修正点
国土交通省のデータセットで一部エラーが出るファイルがあったため、下記の通り修正した。  
■P12-14_12.xml  
ファイルの最後に下記を追加  
``` 
</ksj:TourismResource_Surface>
</ksj:Dataset>
```
■P12-14_17.xml P12-14_21.xml P12-14_22.xml P12-14_28.xml    
半角&を```&amp;```に変更  
``` python
file = open('01_input/P12-14_XX.xml')#XXは変えてください
xml_text = file.read()
file.close()
xml_text = xml_text.replace('&', '&amp;')
f = open('01_input/P12-14_XX_replace.xml', 'w')
f.write(xml_text)
f.close()
``` 